In [3]:
import io
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import pandas as pd
import os

In [7]:
# Pega a pasta do arquivo atual (venditore/)
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Sobe 2 níveis: venditore -> automations -> src -> JamesIA
base_dir = os.path.abspath(os.path.join(current_dir, "../../.."))

# Monta o caminho até secrets/credenciais.json
json_path = os.path.join(base_dir, "secrets", "grand-bridge-465614-h2-af35a1b3a984.json")

In [15]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = service_account.Credentials.from_service_account_file(json_path, scopes=SCOPES)

drive = build('drive', 'v3', credentials=creds)

In [16]:
results = drive.files().list(
    pageSize=10,
    fields="files(id, name, mimeType)"
).execute()

In [17]:
results

{'files': [{'id': '1j-7FGU1WjsGTnoxwqPiiV9aaTtPsMOE5',
   'name': 'venditore-teste',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'id': '1YZx4MYF3AA3XSKrAX0Qj1u40hnuno4O-',
   'name': 'custos04-08-2025.csv',
   'mimeType': 'text/csv'},
  {'id': '11goHCXLdAE6D8FUgbD2OqGVrkXk6OXjt',
   'name': 'custos05-08-2025.csv',
   'mimeType': 'text/csv'}]}

In [24]:
files = results['files']

In [26]:
from datetime import datetime, timedelta

In [28]:
def get_target_filename(files, target_date):
    """
    files: lista retornada pelo Drive
    target_date: datetime.date ou string 'YYYY-MM-DD'
    """
    if isinstance(target_date, str):
        target_date = datetime.strptime(target_date, "%Y-%m-%d").date()

    # Converte para dd-mm-YYYY
    date_str = target_date.strftime("%d-%m-%Y")
    expected_name = f"custos{date_str}.csv"

    for f in files:
        if f["mimeType"] == "text/csv" and f["name"] == expected_name:
            return f  # retorna o dict com id, name...
    return None

In [33]:
file = get_target_filename(files, '2025-08-04')

In [31]:
import tempfile

In [32]:
def download_drive_file_to_temp(drive, file_id: str, suffix=".csv") -> str:
    """Baixa um arquivo do Drive para um arquivo temporário e retorna o caminho."""
    req = drive.files().get_media(fileId=file_id)
    fd, tmp_path = tempfile.mkstemp(suffix=suffix)
    os.close(fd)
    with open(tmp_path, "wb") as f:
        downloader = MediaIoBaseDownload(f, req)
        done = False
        while not done:
            _, done = downloader.next_chunk()
    return tmp_path

In [35]:
tmp_csv_path = download_drive_file_to_temp(drive, file['id'])

In [36]:
from src.automations.venditore.venditore_csv import parse_csv_custos

dados = parse_csv_custos(tmp_csv_path, data_arquivo='2025-08-04')

In [37]:
dados

[{'techprefix': '5006',
  'total_chamadas': 110976,
  'tempo_total_segundos': 3805548,
  'cliente': 'CONTECH',
  'media_chamada': '0 min 34 seg',
  'media_chamada_segundos': 34,
  'valor_total': 2508.09,
  'data': '2025-08-04',
  'empresa': ''},
 {'techprefix': '2131001',
  'total_chamadas': 16984,
  'tempo_total_segundos': 600882,
  'cliente': '(CONTECH) RDF',
  'media_chamada': '0 min 35 seg',
  'media_chamada_segundos': 35,
  'valor_total': 400.59,
  'data': '2025-08-04',
  'empresa': ''},
 {'techprefix': '127001',
  'total_chamadas': 10650,
  'tempo_total_segundos': 359196,
  'cliente': '(CONTECH) QUALITY ASSESSORIA',
  'media_chamada': '0 min 33 seg',
  'media_chamada_segundos': 33,
  'valor_total': 239.46,
  'data': '2025-08-04',
  'empresa': ''},
 {'techprefix': '9004',
  'total_chamadas': 12443,
  'tempo_total_segundos': 418602,
  'cliente': 'CONTECH',
  'media_chamada': '0 min 33 seg',
  'media_chamada_segundos': 33,
  'valor_total': 195.35,
  'data': '2025-08-04',
  'empresa'